### Author: Saeed Abdollahi Taromsari-S5397691

# Import Packages & Initialization

In [1]:
%matplotlib tk

# Python Packages
import sys
import os
import time
from threading import Thread
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 30em; border: 20px solid blue;}</style>"))

# ROS Package"
import rospy
import actionlib
import actionlib.msg

from nav_msgs.msg import Odometry
from geometry_msgs.msg import PoseStamped
from robot_sim.msg import OdoSensor
from sensor_msgs.msg import LaserScan
from robot_sim.msg import PlanningAction, PlanningGoal, PlanningActionFeedback

# Other Packages
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, Label, Text 

# Global Variables
goal_x = 0.0
goal_y = 0.0
robot_x = None
robot_y = None
goals_reached = 0
goals_cancelled = 0
odoMsg = OdoSensor()

client = None
gui = None
counter = 0

update_rate = 100
draw_rate = 10 * update_rate

colors = ["green", "orange", "red"]
regions_ = {
    'right': 0.0,
    'fright': 0.0,
    'front': 0.0,
    'fleft': 0.0,
    'left': 0.0,
}

/tmp/ipykernel_7160/777376939.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Create the User Interface

In [2]:
# Calculate the color for button widgets based on the robot's distance from obstacles
def calculate_color(distance):
    global colors
    i = 0
    if distance >3.0:
        i = 0
    if (distance>1.0 and distance<=3.0):
        i = 1
    if distance <=1.0:
        i = 2
    return colors[i]

# Calcualte the minimum distance of the robot from the obstacles
def find_minimum_distance():
    global regions_
    
    min_distance = 100000
    for key in regions_.keys():
        if regions_[key] < min_distance:
            min_distance = regions_[key]
    return min_distance
    
# Create Navigation Control Button
send_goal = Button(description ='Send Goal')
send_goal.style = ButtonStyle(button_color='yellow')
send_goal.layout = Layout(width='auto', height="20%", margin="20px")
             
# Create Navigation Control TextBox
x_goal = Text()
x_goal_container = HBox([Label(value='Goal X'), x_goal ], layout=Layout(display='flex', flex_flow='row', justify_content='space-between'))

y_goal = Text()
y_goal_container = HBox([Label(value='Goal Y'), y_goal], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
# Create Navigation Information Textboxes
x_position = Text(placeholder='Unknown', description='Position X:', disabled=True)
x_position_container = HBox([x_position ], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
y_position = Text(placeholder='Unknown', description='Position Y:', disabled=True)
y_position_container = HBox([y_position], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
obstacle_distance = Text(placeholder='Unknown', description='Min Distance:', disabled=True)
obstacle_distance_container = HBox([obstacle_distance], layout=Layout(display='flex', flex_flow='row', justify_content='space-between', state='disabled'))
        
# Create Obstacle Visualizer Bottons
obstacle_left_button = Button(description='Left')
obstacle_left_button.layout = Layout(width='auto' , height="auto")
obstacle_left_button.style = ButtonStyle(button_color=colors[0])
        
obstacle_front_left_button = Button(description='Front-Left')
obstacle_front_left_button.layout = Layout(width='auto' , height="auto")
obstacle_front_left_button.style = ButtonStyle(button_color=colors[0])
        
Robot_button = Button(description='Robot')
Robot_button.layout = Layout(width='auto' , height="auto")
Robot_button.style = ButtonStyle(button_color='yellow')
        
obstacle_front_button = Button(description='Front')
obstacle_front_button.layout = Layout(width='auto' , height="auto")
obstacle_front_button.style = ButtonStyle(button_color=colors[0])
        
obstacle_front_right_button = Button(description='Front-Right')
obstacle_front_right_button.layout = Layout(width='auto' , height="auto")
obstacle_front_right_button.style = ButtonStyle(button_color=colors[0])
        
obstacle_right_button = Button(description='Right')
obstacle_right_button.layout = Layout(width='auto' , height="auto")
obstacle_right_button.style = ButtonStyle(button_color=colors[0])
        
# Container Widgets        
navigation_box_label = Label('Navigation Control:')
navigation_box = VBox([navigation_box_label, x_goal_container, y_goal_container, send_goal])
navigation_box.layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid', width='50%', padding="5px")
        
monitoring_box = VBox([Label('Kinematic Monitoring:'), x_position_container, y_position_container, Label('Obstacle Distance:'), obstacle_distance_container])
monitoring_box.layout = Layout(border='solid', width='50%',padding="10px")
        
control_box = HBox([navigation_box, monitoring_box])
control_box.layout = Layout(width='100%', height='100%')

obstacle_visual_box_label = Label('Obstacle Detectors: GREEN (distance>3) - ORANGE (1<distance<3) - RED (distance<1)')
obstacle_visual_box_visual = GridBox(children=[obstacle_front_left_button, obstacle_front_button, obstacle_front_right_button, obstacle_left_button, Robot_button, obstacle_right_button])
obstacle_visual_box_visual.layout = Layout(
            width='100%', height='100%',
            grid_template_columns='33% 33% 33%',
            grid_template_rows='auto auto auto',
            grid_gap='5px 10px',
            padding="20px 20px 20px 20px")
       
obstacle_visual_box = VBox([obstacle_visual_box_label, obstacle_visual_box_visual])
obstacle_visual_box.layout = Layout(display='flex', flex_flow='column', align_items='stretch', border='solid', width='100%')   
obstacle_box = HBox([obstacle_visual_box])

# Main Widget
main_widget = VBox([control_box, obstacle_box])

# Draw Charts
class OdoVisualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.fig.set_figwidth(4)
        self.fig.set_figheight(3)
        self.ln, = plt.plot([], [], marker = 'o', color='r')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        # Style the Plot
        plt.style.use('fast')
        plt.xlabel("X Position", fontsize=15, color='blue', fontweight='bold')
        plt.ylabel("Y Position", fontsize=15, color='blue', fontweight='bold')
        self.ax.set_title("Robot & Goal Position", color = 'blue', fontweight='bold')
        # Style the grid.
        self.ax.grid(which='major', color='blue', linewidth=1)
        self.ax.grid(which='major', color='blue', linewidth=1)
     
        # Adjust the axis
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def odom_callback(self, x, y):
        self.x_data.append(x)
        self.y_data.append(y)
    
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

class GoalVisualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.fig.set_figwidth(4)
        self.fig.set_figheight(3)
        self.goals_status = ['Reached', 'Cancelled']
        self.gaols_value = [0, 0]
       
    def plot_init(self):
        # Style the Plot
        plt.style.use('fast')
        plt.xlabel("Goals", fontsize=15, color='blue', fontweight='bold')
        plt.ylabel("Numbers", fontsize=15, color='blue', fontweight='bold')
        self.ax.set_title("Reached/Cancelled", color = 'blue', fontweight='bold')

    def goal_callback(self, reached, cancelled):
        self.gaols_value = [reached, cancelled]
        
    def update_plot(self, frame):
        self.ax.cla()
        self.ax.bar(self.goals_status, self.gaols_value, color=['green', 'red'], width = 0.8)
        
vis2 = GoalVisualiser()
ani2 = FuncAnimation(vis2.fig, vis2.update_plot, init_func=vis2.plot_init, interval=draw_rate)

vis1 = OdoVisualiser()
ani1 = FuncAnimation(vis1.fig, vis1.update_plot, init_func=vis1.plot_init, interval=draw_rate)

# Update GUI
def updateGUI():
    global robot_x, robot_y, vis
    x_position.value = str(robot_x)
    y_position.value = str(robot_y)    
    obstacle_left_button.style = ButtonStyle(button_color=calculate_color(regions_['left']))
    obstacle_front_left_button.style = ButtonStyle(button_color=calculate_color(regions_['fleft']))
    obstacle_front_button.style = ButtonStyle(button_color=calculate_color(regions_['front']))
    obstacle_front_right_button.style = ButtonStyle(button_color=calculate_color(regions_['fright']))
    obstacle_right_button.style = ButtonStyle(button_color=calculate_color(regions_['right']))
    obstacle_distance.value = str(find_minimum_distance())
    vis1.odom_callback(robot_x, robot_y)

# Set_New_Goal Button Click Handler 
def handle_send_goal_click(event):
    global goal_x, goal_y, client
    goal_x = x_goal.value
    goal_y = y_goal.value
    circle1 = plt.Circle((float(goal_x), float(goal_y)), 0.5, color='g')
    vis1.ax.add_patch(circle1)
    sendGoal(client, float(goal_x), float(goal_y))
    
# Add event listeners to the Navigation Control
send_goal.on_click(handle_send_goal_click) 

# Display the GUI
display(main_widget)

# Initialize the GUI
updateGUI()

# Run the ROS Application

In [3]:
# Create a Cloent for Action Server
def createClient():
    try:#Trys to create an action-client
        #Creates an action-client
        client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    except rospy.ROSInterruptException:#If th process failed, show an error message
        print("Could not create the client", file=sys.stderr) 
    return client

# Send Goal to the Action Server
def sendGoal(client, x, y):
    target_pose = PoseStamped()
    
    try:#Trys to create a goal and snd it to the action-server
        # Waits until the action server has started up and started listening for goals
        client.wait_for_server()
        
       #Creates a goal to send to the action server.
        target_pose.pose.position.x = x
        target_pose.pose.position.y = y       
        goal = PlanningGoal(target_pose)

        #Sends the goal to the action server
        client.send_goal(goal)
    except rospy.ServiceException as e: #If the process failed, show an error message
        print("Service call failed: %s"%e)

# Receive the Odometry Data
def setOdoMessage(msg):
    global robot_x, robot_y, counter, trajectory_x, trajectory_y
    #Fills the custom odoMsg object by the odemetry data
    robot_x = msg.pose.pose.position.x
    robot_y = msg.pose.pose.position.y
 
    counter +=1 
    if counter == update_rate:
        updateGUI()
        counter = 0

# Receive the Laser Scanner Data
def clbk_laser(msg):
    global regions_, counter
    regions_ = {
        'right':  min(min(msg.ranges[0:143]), 10),
        'fright': min(min(msg.ranges[144:287]), 10),
        'front':  min(min(msg.ranges[288:431]), 10),
        'fleft':  min(min(msg.ranges[432:575]), 10),
        'left':   min(min(msg.ranges[576:713]), 10),
    }
    if counter == update_rate:
        updateGUI()

# Receive the Goal Summary Data
def updateGoalSummary(msg):
    global goals_reached
    global goals_cancelled

    #Check the state value of the 'PlanningActionFeedback' message
    state = msg.feedback.stat
    if state == "Target reached!":#Update the goals_reached
        goals_reached = goals_reached + 1

    if state == "Target cancelled!":#Update the goals_cancelled
        goals_cancelled = goals_cancelled + 1  
        
    vis2.goal_callback(goals_reached, goals_cancelled)

# The main function of ROS application
def main():
    global client
    
    # Create the GUI
    rospy.init_node('jupyter_node')

    # Create a ROS Client
    client = createClient()

    #Creates a subscriber to receive the 'Odometry' message
    odoMessageSubscriber = rospy.Subscriber("odom", Odometry, setOdoMessage)
    
    #Creates a subscriber to receive the 'PlanningActionFeedback' message
    goalSummarySubscriber = rospy.Subscriber("reaching_goal/feedback", PlanningActionFeedback, updateGoalSummary)

    # Creates a subscriber to receive the 'Laser' message
    sub = rospy.Subscriber('/scan', LaserScan, clbk_laser)
      
if __name__ == "__main__":
    main()
